### 구글 드라이브 마운트 및 작업 폴더 설정

In [81]:
from google.colab import drive
import os

# 구글 드라이브 마운트
drive.mount('/content/drive')

# 작업 디렉토리 설정 (구글 드라이브의 "project" 폴더로 이동)
PROJECT_DIR = "/content/drive/My Drive/project"
CONTENT_DIR = os.path.join(PROJECT_DIR, "content")
STYLE_DIR = os.path.join(PROJECT_DIR, "style")
RESULTS_DIR = os.path.join(PROJECT_DIR, "results")

# 결과 저장 폴더가 없으면 생성
os.makedirs(RESULTS_DIR, exist_ok=True)

print("Google Drive 마운트 완료!")
print(f"Content 폴더: {CONTENT_DIR}")
print(f"Style 폴더: {STYLE_DIR}")
print(f"Results 저장 폴더: {RESULTS_DIR}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive 마운트 완료!
📂 Content 폴더: /content/drive/My Drive/project/content
📂 Style 폴더: /content/drive/My Drive/project/style
📂 Results 저장 폴더: /content/drive/My Drive/project/results


### content 폴더의 이미지 3개를 style 폴더의 각 스타일 이미지와 페어로 연결


In [84]:
# 콘텐츠 이미지 리스트
content_images = [
    "helmet.png",
    "Log Cabin.png",
    "A beautiful young woman.png",
    "Alley after rain.png",
    "Clean house building.png",
    "boat.png"
]

# 스타일 이미지 리스트 (각 폴더에서 1장씩 선택)
style_images = [
    "Popart/00.jpg",
    "Watercolor/00.jpg",
    "Analog/00.jpg",
    "Cyberpunk/00.jpg",
    "MineCraft/00.jpg",
    "Fauvism/00.jpg"
]

# 콘텐츠와 스타일 이미지 페어 생성
image_pairs = list(zip(content_images, style_images))

# 🔍 페어링 확인
for content, style in image_pairs:
    content_path = os.path.join(CONTENT_DIR, content)
    style_path = os.path.join(STYLE_DIR, style)
    print(f"🖼 Content Image Path: {content_path}")
    print(f"🎨 Style Image Path: {style_path}")

🖼 Content Image Path: /content/drive/My Drive/project/content/helmet.png
🎨 Style Image Path: /content/drive/My Drive/project/style/Popart/00.jpg
🖼 Content Image Path: /content/drive/My Drive/project/content/Log Cabin.png
🎨 Style Image Path: /content/drive/My Drive/project/style/Watercolor/00.jpg
🖼 Content Image Path: /content/drive/My Drive/project/content/A beautiful young woman.png
🎨 Style Image Path: /content/drive/My Drive/project/style/Analog/00.jpg
🖼 Content Image Path: /content/drive/My Drive/project/content/Alley after rain.png
🎨 Style Image Path: /content/drive/My Drive/project/style/Cyberpunk/00.jpg
🖼 Content Image Path: /content/drive/My Drive/project/content/Clean house building.png
🎨 Style Image Path: /content/drive/My Drive/project/style/MineCraft/00.jpg
🖼 Content Image Path: /content/drive/My Drive/project/content/boat.png
🎨 Style Image Path: /content/drive/My Drive/project/style/Fauvism/00.jpg


#### 사용한 스타일
- Popart
- Watercolor
- Analog
- Cyberpunk
- MineCraft
- Fauvism

스타일이 잘 어울릴 것 같은 이미지를 선정하여 컨텐츠 이미지와 페어를 생성

### 이미지 로딩 및 전처리

In [85]:
# 이미지 전처리를 위한 변환 정의
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # 🔥 스타일 대비 증가
])

# 이미지 로딩 함수
def load_image(image_path, transform, device):
    image = Image.open(image_path).convert("RGB")  # 이미지 로드 및 RGB 변환
    image = transform(image).unsqueeze(0)  # 배치 차원 추가 (1, C, H, W)
    return image.to(device)  # GPU 또는 CPU로 이동

# 콘텐츠 및 스타일 이미지 경로 설정
content_paths = [os.path.join(CONTENT_DIR, img) for img in content_images]
style_paths = [os.path.join(STYLE_DIR, img) for img in style_images]

# 이미지 로드
content_images_tensors = [load_image(path, transform, device) for path in content_paths]
style_images_tensors = [load_image(path, transform, device) for path in style_paths]

# 🔍 전처리된 이미지의 평균값 및 크기 확인
for idx, img_tensor in enumerate(content_images_tensors):
    print(f"🖼 Content Image {idx} - Tensor Mean: {img_tensor.mean().item()}, Shape: {img_tensor.shape}")

for idx, img_tensor in enumerate(style_images_tensors):
    print(f"🎨 Style Image {idx} - Tensor Mean: {img_tensor.mean().item()}, Shape: {img_tensor.shape}")

# 확인
print(f"✅ 총 {len(content_images_tensors)} 개의 콘텐츠 이미지 로드 완료!")
print(f"✅ 총 {len(style_images_tensors)} 개의 스타일 이미지 로드 완료!")

🖼 Content Image 0 - Tensor Mean: -0.49436163902282715, Shape: torch.Size([1, 3, 512, 512])
🖼 Content Image 1 - Tensor Mean: 0.46989545226097107, Shape: torch.Size([1, 3, 512, 512])
🖼 Content Image 2 - Tensor Mean: 0.03280900418758392, Shape: torch.Size([1, 3, 512, 512])
🖼 Content Image 3 - Tensor Mean: -0.12601706385612488, Shape: torch.Size([1, 3, 512, 512])
🖼 Content Image 4 - Tensor Mean: 0.19021904468536377, Shape: torch.Size([1, 3, 512, 512])
🖼 Content Image 5 - Tensor Mean: 0.42276835441589355, Shape: torch.Size([1, 3, 512, 512])
🎨 Style Image 0 - Tensor Mean: -0.035296354442834854, Shape: torch.Size([1, 3, 512, 512])
🎨 Style Image 1 - Tensor Mean: 0.41975468397140503, Shape: torch.Size([1, 3, 512, 512])
🎨 Style Image 2 - Tensor Mean: -0.07787813991308212, Shape: torch.Size([1, 3, 512, 512])
🎨 Style Image 3 - Tensor Mean: -0.4024043083190918, Shape: torch.Size([1, 3, 512, 512])
🎨 Style Image 4 - Tensor Mean: -0.36743542551994324, Shape: torch.Size([1, 3, 512, 512])
🎨 Style Image 

초기에는 VGG-19 모델의 학습 시 사용된 정규화 값
Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
을 적용했으나, 스타일이 제대로 반영되지 않는 문제가 발생하여 정규화 값을 수정함

변경된 정규화 값:
Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])

스타일 정보를 극대화하여 컨텐츠 이미지에 반영을 통해 스타일 반영 문제가 해결된 것으로 추측

### VGG-19 모델 로드 및 특징 추출을 위한 준비

In [87]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt

# VGG-19 모델 로드 (사전 학습된 모델 사용)
vgg = models.vgg19(pretrained=True).features

# 모델을 평가 모드로 설정 (파라미터 업데이트 X)
vgg.eval()

# GPU 사용 가능하면 GPU로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg.to(device)

print("VGG-19 모델 로드 완료!")

# 콘텐츠와 스타일을 추출할 레이어 인덱스 정의
content_layers = ["conv_4"]   # 콘텐츠 특징 추출
style_layers = ["conv_1", "conv_2", "conv_3", "conv_4", "conv_5"]  # 스타일 특징 추출

# VGG-19의 레이어 매핑 (PyTorch에서는 Sequential 구조라서 직접 매핑 필요)
vgg_layers = {
    "0": "conv_1", "5": "conv_2", "10": "conv_3", "19": "conv_4", "28": "conv_5"
}


print("특징 추출을 위한 레이어 설정 완료!")

✅ VGG-19 모델 로드 완료!
✅ 특징 추출을 위한 레이어 설정 완료!


- VGG-19 모델은 ImageNet 데이터셋으로 학습된 사전 학습 모델(Pretrained Model)  

- 학습이 아닌 특징 추출에 사용되므로 eval()을 호출하여 BatchNorm, Dropout 등의 영향 제거


#####컨텐츠 및 스타일 특징을 추출할 레이어 지정
    - 콘텐츠 특징은 conv_4에서 추출 (이미지의 주요 구조 유지)
    - 스타일 특징은 conv_1 ~ conv_5에서 추출 (스타일 질감, 색감 정보 활용)


### VGG-19 특징 추출 클래스

In [88]:
import torch.nn as nn

class VGGFeatureExtractor(nn.Module):
    def __init__(self, model, content_layers, style_layers, vgg_layers):
        super(VGGFeatureExtractor, self).__init__()

        self.model = model  # VGG-19 모델
        self.content_layers = content_layers  # 콘텐츠 특징을 추출할 레이어
        self.style_layers = style_layers  # 스타일 특징을 추출할 레이어
        self.layers = content_layers + style_layers  # 모든 추출 레이어 리스트
        self.vgg_layers = vgg_layers

    def forward(self, x):
        content_features = {}
        style_features = {}

        for name, layer in self.model._modules.items():
            x = layer(x)

            if name in self.vgg_layers:
                layer_name = self.vgg_layers[name]

                if layer_name in self.content_layers:
                    content_features[layer_name] = x

                if layer_name in self.style_layers:
                    style_features[layer_name] = x

        return content_features, style_features

입력 이미지x를 VGG-19 모델의 각 레이어를 통과  
지정된 레이어에 도달하면 해당 출력을 저장
- conv_4: 콘텐츠 특징 저장
- conv_1 ~ conv_5: 스타일 특징 저장



### 특징 추출기를 이용한 특징 추출

In [89]:
# 특징 추출기 객체 생성
feature_extractor = VGGFeatureExtractor(vgg, content_layers, style_layers, vgg_layers).to(device)

# 콘텐츠 및 스타일 특징 저장 딕셔너리
content_features_list = []
style_features_list = []

# 콘텐츠 & 스타일 이미지별로 특징 추출 (🚀 각 이미지 개별적으로 처리)
for idx, (content_img, style_img) in enumerate(zip(content_images_tensors, style_images_tensors)):

    # ✅ 콘텐츠 이미지 특징 추출 (🚀 detach 추가)
    content_features, _ = feature_extractor(content_img)
    content_features = {k: v.detach() for k, v in content_features.items()}  # 🔥 detach() 추가
    print(f"🖼 Content Image {idx} Feature Map Mean (conv_4): {content_features['conv_4'].mean().item()}")

    # ✅ 스타일 이미지 특징 추출 (🚀 detach 추가)
    _, style_features = feature_extractor(style_img)
    style_features = {k: v.detach() for k, v in style_features.items()}  # 🔥 detach() 추가
    print(f"🎨 Style Image {idx} Feature Map Mean (conv_4): {style_features['conv_4'].mean().item()}")

    # ✅ 특징 저장
    content_features_list.append(content_features)
    style_features_list.append(style_features)

print(f"✅ 총 {len(content_features_list)} 개의 콘텐츠 특징 추출 완료!")
print(f"✅ 총 {len(style_features_list)} 개의 스타일 특징 추출 완료!")

🖼 Content Image 0 Feature Map Mean (conv_4): 0.8290873765945435
🎨 Style Image 0 Feature Map Mean (conv_4): 1.9516329765319824
🖼 Content Image 1 Feature Map Mean (conv_4): 0.5701032280921936
🎨 Style Image 1 Feature Map Mean (conv_4): 0.4203479290008545
🖼 Content Image 2 Feature Map Mean (conv_4): 0.5154279470443726
🎨 Style Image 2 Feature Map Mean (conv_4): 0.7498735189437866
🖼 Content Image 3 Feature Map Mean (conv_4): 0.9821031093597412
🎨 Style Image 3 Feature Map Mean (conv_4): 1.0774731636047363
🖼 Content Image 4 Feature Map Mean (conv_4): 0.8152661323547363
🎨 Style Image 4 Feature Map Mean (conv_4): 0.8888296484947205
🖼 Content Image 5 Feature Map Mean (conv_4): 0.7285125255584717
🎨 Style Image 5 Feature Map Mean (conv_4): 0.8332745432853699
✅ 총 6 개의 콘텐츠 특징 추출 완료!
✅ 총 6 개의 스타일 특징 추출 완료!


#####특이사항
처음 스타일 전이를 진행할 때, 손실(loss) 값이 0으로 나오는 현상이 발생했음.  

콘텐츠 이미지와 스타일 이미지가 각각의 역할을 수행하지 않음

	- 콘텐츠 이미지는 콘텐츠 특징을 추출하는 레이어에서만
	- 스타일 이미지는 스타일 특징을 추출하는 레이어에서만 작동해야 함
	- 하지만 처음 코드에서는 두 이미지가 동일한 경로를 거치며 특징이 추출됨 → 결과적으로 같은 값이 저장됨

특징 추출기가 각각의 이미지에 대해 독립적으로 동작하지 않음

	- 따라서 콘텐츠 이미지와 스타일 이미지에서 동일한 특징맵 값이 나옴
	- 그 결과 스타일 손실과 콘텐츠 손실이 0으로 계산됨

##### 해결방법
	- feature_extractor(content_img)와 feature_extractor(style_img)를 각각 독립적으로 실행
	- 콘텐츠 특징과 스타일 특징을 별도의 딕셔너리에 저장하여 분리
    - vgg_layers를 활용하여 정확한 레이어에서만 특징을 추출
	- 콘텐츠와 스타일 특징을 따로 저장하도록 forward() 함수 수정

### 스타일 전이를 위한 손실 함수 정의

In [90]:
import torch.nn.functional as F

# ✅ Gram Matrix 계산 (정규화 방식 수정)
def compute_gram_matrix(feature_map):
    _, C, H, W = feature_map.shape
    feature_map = feature_map.view(C, H * W)  # (C, N)
    gram_matrix = torch.mm(feature_map, feature_map.t())  # (C, C)

    # 🔥 정규화 방식 수정: 더 강한 스타일 손실 반영
    return gram_matrix / (H * W)  # 기존의 (C * H * W) ** 2 에서 변경

# ✅ 콘텐츠 손실 함수
def compute_content_loss(gen_features, content_features):
    return F.mse_loss(gen_features, content_features.detach())  # 🔥 detach() 제거


# ✅ 스타일 손실 함수
def compute_style_loss(gen_features, style_features, style_layers):
    loss = 0
    for layer in style_layers:
        gen_gram = compute_gram_matrix(gen_features[layer])
        style_gram = compute_gram_matrix(style_features[layer].detach())
        loss += F.mse_loss(gen_gram, style_gram)  # 🔥 .detach() 사용 X
    return loss / len(style_layers)

# ✅ 생성 이미지 초기화 (옵션: 콘텐츠 기반 vs 랜덤 노이즈)
def initialize_generated_image(content_img, use_noise=True):
    if use_noise:
        generated_img = torch.randn_like(content_img).to(content_img.device) * 0.1  # 랜덤 노이즈 초기화
    else:
        generated_img = content_img.clone().detach()
        generated_img += torch.randn_like(generated_img) * 0.01  # 콘텐츠 이미지 기반 노이즈 추가
    return generated_img.requires_grad_(True)

스타일 전이는 두 가지 손실을 최소화하는 방식으로 학습이 진행됨  

    - 콘텐츠 손실 (Content Loss): 생성 이미지와 콘텐츠 이미지의 특징맵이 유사하도록 만듦  
    - 스타일 손실 (Style Loss): 생성 이미지와 스타일 이미지의 스타일 정보(패턴 및 질감)를 유사하게 만듦

**콘텐츠 손실**

    - 콘텐츠 손실은 MSE 손실을 사용하여 생성 이미지의 특징맵이 콘텐츠 이미지의 특징맵과 유사하도록 유도
    - detach()를 사용하여 content_features가 역전파에 영향을 주지 않도록 설정

**스타일 손실**

	- 스타일 손실은 Gram Matrix를 활용한 MSE 손실을 사용
	- 각 스타일 레이어에서 생성 이미지와 스타일 이미지의 Gram Matrix를 계산
	- Gram Matrix는 특징맵의 채널 간 상관관계(스타일 패턴)를 캡처하는 역할을 함
	- .detach()를 사용하여 style_features가 업데이트되지 않도록 설정

### 특이사항

**Gram Matrix 정규화 방식 변경**  

원래는 Gram Matrix를 채널(C), 높이(H), 너비(W) 값의 제곱으로 정규화하였으나 스타일 손실 값이 너무 작아지는 문제가 발생

##### 해결방법  
	- 채널 수(C)를 제외하고 높이(H)와 너비(W)만 사용하여 정규화
	- 이렇게 하면 스타일 손실이 더 강하게 반영되어 스타일 전이가 더욱 뚜렷하게 적용됨

**생성 이미지 초기화 방식변경**

기존에는 content_img.clone()을 사용하여 콘텐츠 이미지를 기반으로 생성 이미지 초기화

##### 해결방법

`use_noise=True` 옵션 설정   
스타일이 강하게 반영되지 않는 문제가 발생하여 완전한 랜덤 노이즈로 초기화하는 옵션을 추가  

**-> 위 두가지 방법을 통해 스타일 반영이 잘 안되던 문제를 효과적으로 개선하였음**

### 스타일 전이 최적화 코드

In [91]:
# 스타일 전이 설정
num_steps = 1000  # 최적화 반복 횟수
content_weight = 1e3  # 1,000
style_weight = 1e5 # 10,000
learning_rate = 0.05

# 결과 저장 경로
os.makedirs(RESULTS_DIR, exist_ok=True)

# 스타일 전이 실행
for idx, (content_img, style_img) in enumerate(zip(content_images_tensors, style_images_tensors)):

    content_path = content_paths[idx]
    style_path = style_paths[idx]

    print(f"\n스타일 전이 진행 중: {idx + 1}/{len(content_images_tensors)}")
    print(f"Content Image Path: {content_path}")
    print(f"Style Image Path: {style_path}\n")


    # 생성 이미지 초기화 (콘텐츠 이미지 기반, 약간의 노이즈 추가)
    generated_img = initialize_generated_image(content_img, use_noise=False)

    # 콘텐츠 및 스타일 특징을 추출
    content_features, _ = feature_extractor(content_img)
    _, style_features = feature_extractor(style_img)

    # 최적화 설정
    optimizer = torch.optim.Adam([generated_img], lr=learning_rate)

    # 스타일 전이 학습 진행
    for step in range(num_steps):
        optimizer.zero_grad()

        # 생성된 이미지의 특징 추출
        gen_content_features, gen_style_features = feature_extractor(generated_img)

        # 콘텐츠 손실 계산
        content_loss = compute_content_loss(gen_content_features["conv_4"], content_features["conv_4"])

        # 스타일 손실 계산
        style_loss = compute_style_loss(gen_style_features, style_features, style_layers)

        # 총 손실 계산
        total_loss = content_weight * content_loss + style_weight * style_loss

        # 100 스텝마다 손실 값만 출력
        if step % 100 == 0:
            print(f"Step {step} - Content Loss: {content_loss.item()}, Style Loss: {style_loss.item()}, Total Loss: {total_loss.item()}")

        # 손실 값이 0이면 경고 메시지 출력
        if total_loss.item() == 0:
            print(f"Step {step}: Loss is zero. Something is wrong!")
            continue

        # 역전파 수행
        total_loss.backward()

        # 이미지 업데이트
        optimizer.step()

    # 최적화된 이미지 저장
    output_img = generated_img.detach().cpu().squeeze().permute(1, 2, 0).numpy()
    output_img = (output_img - output_img.min()) / (output_img.max() - output_img.min())  # 정규화
    output_img = (output_img * 255).astype("uint8")

    # 결과 저장
    result_path = os.path.join(RESULTS_DIR, f"stylized_{idx + 1}.png")
    Image.fromarray(output_img).save(result_path)
    print(f"스타일 전이 완료! 결과 저장: {result_path}")
    print(f"다음 이미지 스타일 전이 진행 중...\n")

print("모든 스타일 전이 완료!")


🎨 스타일 전이 진행 중: 1/6
🖼 Content Image Path: /content/drive/My Drive/project/content/helmet.png
🎨 Style Image Path: /content/drive/My Drive/project/style/Popart/00.jpg

Step 0 - Content Loss: 0.07060706615447998, Style Loss: 32.96088790893555, Total Loss: 3296159.25
Step 100 - Content Loss: 25.46147346496582, Style Loss: 0.5335103869438171, Total Loss: 78812.515625
Step 200 - Content Loss: 25.28717803955078, Style Loss: 0.2681983411312103, Total Loss: 52107.01171875
Step 300 - Content Loss: 25.251018524169922, Style Loss: 0.18662133812904358, Total Loss: 43913.15234375
Step 400 - Content Loss: 24.908599853515625, Style Loss: 0.1465000957250595, Total Loss: 39558.609375
Step 500 - Content Loss: 24.439102172851562, Style Loss: 0.1312202364206314, Total Loss: 37561.125
Step 600 - Content Loss: 24.499135971069336, Style Loss: 0.11110204458236694, Total Loss: 35609.33984375
Step 700 - Content Loss: 25.01008415222168, Style Loss: 0.10574418306350708, Total Loss: 35584.5
Step 800 - Content Loss:

####학습 과정

콘텐츠 및 스타일 이미지의 경로 출력  
    
    - 현재 스타일 전이 중인 콘텐츠 및 스타일 이미지 경로를 출력하여 확인


생성 이미지 초기화

    - 콘텐츠 이미지 기반으로 초기 생성 이미지를 설정
	- 약간의 랜덤 노이즈를 추가하여 더욱 다양한 변화를 유도

콘텐츠 및 스타일 특징 추출

	- feature_extractor(content_img): 콘텐츠 이미지의 특징맵 추출
	- feature_extractor(style_img): 스타일 이미지의 특징맵 추출

스타일 전이 최적화 진행

	- Adam 옵티마이저를 사용하여 학습 진행
	- num_steps 만큼 반복하여 콘텐츠 및 스타일 손실을 계산하고 최적화 수행
	- 100 스텝마다 손실 값(content_loss, style_loss, total_loss)을 출력하여 학습 과정 모니터링
	- 최종적으로 스타일 전이된 이미지를 저장

#### 특이사항
논문에서 사용한 L-BFGS (Limited-memory Broyden-Fletcher-Goldfarb-Shanno, 제한된 메모리를 사용하는 BFGS) 방법은 2차 최적화 기법으로, 이전 기울기 정보를 활용하여 더 빠르게 수렴하도록 설계된 알고리즘

	- L-BFGS는 Hessian 행렬(2차 미분 행렬)을 근사하여 최적화하는 방식으로, 메모리를 적게 사용하면서도 더 빠르고 안정적인 수렴을 기대할 수 있음

**Adam 옵티마이저**
1. 미니배치 학습 가능
2. 적응형 학습률 조정
3. 빠른 수렴

**-> 현재 환경에선 간단하게 구현하였기에 Adam 옵티마이저를 사용하기로 결정**  

####가중치 설정 변경

**초기설정**
```
content_weight = 1e4  # 10,000
style_weight = 1e2  # 100
learning_rate = 0.03
```
→ 문제: 스타일이 거의 반영되지 않음

**설정 변경**
```
content_weight = 1e3  # 1,000
style_weight = 1e5  # 100,000
learning_rate = 0.05
```
→ 변경 이유:

- 콘텐츠 가중치를 낮추고 스타일 가중치를 높여 스타일이 더 강하게 반영되도록 조정

- 학습률을 0.05로 증가하여 더 빠르게 수렴하도록 설정

이러한 변경을 통해 스타일 손실이 더 잘 반영되었으며, 최종적으로 만족스러운 스타일 전이 결과를 얻음

결괴는 폴더에 압축 파일로 제출